In [36]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm

from bs4 import BeautifulSoup


In [37]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


### Webscraping

In [38]:
# url
url = "https://open.pluralpolicy.com/data/session-csv/"

In [39]:
#webscraping set up

# Path to your WebDriver executable (adjust if necessary)
webdriver_path = r"C:\Users\clutz\hunt_env\chrome driver\chromedriver-win64\chromedriver.exe"

# Set up Chrome options
chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run in headless mode (no GUI)

# Set up WebDriver service
service = Service(webdriver_path)

# Initialize WebDriver
driver = webdriver.Chrome(service=service, options=chrome_options)
driver.get(url)

In [40]:
# Wait for the page to load and the tbody to be present
wait = WebDriverWait(driver, 20)

#click login
login = driver.find_elements(By.TAG_NAME, "a")
# print(login.text)
for l in login:
    print(l.text)
    if 'log in' in l.text.lower():
        
        login_click = l
        break


        # print(str(l.get_attribute('href')))

login_click.click()


#log in
driver.find_element(By.XPATH,'//*[@id="id_login"]').send_keys("cnlutz20@gmail.com")
driver.find_element(By.XPATH,'//*[@id="id_password"]').send_keys("4Ducksquack&")
signin = driver.find_element(By.XPATH,'/html/body/div/div/div/form/button')
print(signin.text)
print(type(signin))
signin.click()


Log In
Sign In
<class 'selenium.webdriver.remote.webelement.WebElement'>


In [41]:
#states regex set up
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", 
    "Connecticut", "Delaware", "Florida", "Georgia", "Hawaii", "Idaho", 
    "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", 
    "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", 
    "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", 
    "New Hampshire", "New Jersey", "New Mexico", "New York", 
    "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", 
    "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", 
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", 
    "West Virginia", "Wisconsin", "Wyoming"
]

states_reg_pat = re.compile("|".join(states))
# print(states_reg_pat)



In [42]:
#getting csvs
driver.get(url)

csvs = driver.find_elements(By.TAG_NAME,'a')

dfs = []
for c in csvs:
    element_text = c.text  # Get the text content of the element
    if re.search(states_reg_pat, element_text):
        # print(f"Match found: {element_text}")
        x = re.match(states_reg_pat, element_text)
        state = x.group()

        description = re.split(states_reg_pat, element_text)[-1].strip()
        link =c.get_attribute('href')
        
        df = pd.DataFrame([[state, description, link]], columns=['state', "description", "link"])
        # df = pd.DataFrame({"state": [state], "description":[description], "link":[link]})
        dfs.append(df)
csv_dfs = pd.concat(dfs).reset_index(drop = True)


### Data Manipulation

In [43]:
#find records to extract
csv_dfs['fetch_1_25'] = False
# outlier_states = []
for i,j in enumerate(csv_dfs['description']):
    if re.search(r'2025|2024', str(j)):
        # print(j)
        csv_dfs.loc[i,'fetch_1_25'] = True
    # else:
        # outlier_states.append(csv_dfs.loc[i,'state'])
csv_dfs = csv_dfs.reset_index(drop=True)

In [44]:
#find outlier states
grouped = csv_dfs.groupby(['state']).agg({
    'fetch_1_25': lambda x: " ".join(map(str, set(x)))
}).reset_index()

# csv_dfs

# print(grouped.first())
# grouped = pd.concat(map(lambda x: x[1], grouped))
grouped_r = grouped.groupby(['fetch_1_25']).agg({
    'state': lambda x: list(set(x))
}).reset_index()


outlier_states = grouped_r.loc[0,'state']


print(csv_dfs.columns)
o_dfs = []
for o in outlier_states:
    o_vals = csv_dfs[csv_dfs['state'] == o]
    print(o_vals.to_string())
    o_vals = o_vals.tail(3)
    print(o_vals.to_string())
    o_dfs.append(o_vals)

outliers = pd.concat(o_dfs)


Index(['state', 'description', 'link', 'fetch_1_25'], dtype='object')
        state            description                                                                                    link  fetch_1_25
174  Illinois  100th Regular Session           https://data.openstates.org/csv/latest/IL_100th_csv_pK3Mu6NLMQBcUWiXG6NnY.zip       False
175  Illinois  100th Special Session  https://data.openstates.org/csv/latest/IL_100th-special_csv_4kaqpIAZNl8bDTDhK3ZtWq.zip       False
176  Illinois  101st Regular Session          https://data.openstates.org/csv/latest/IL_101st_csv_4gtPEmbdo1CAxflrmoKzPb.zip       False
177  Illinois  102nd Regular Session          https://data.openstates.org/csv/latest/IL_102nd_csv_10XD9oCU9WyQEqFi9UKElw.zip       False
178  Illinois  103rd Regular Session          https://data.openstates.org/csv/latest/IL_103rd_csv_1zlH2WjfhijmUdSgoBD8UK.zip       False
179  Illinois  104th Regular Session          https://data.openstates.org/csv/latest/IL_104th_csv_2IVMDJvIhb

In [ ]:
#final df compilation
csv_dfs = csv_dfs[csv_dfs['fetch_1_25'] == True]

download_data = pd.concat([csv_dfs,outliers])
download_data.sort_values("state", inplace=True)
download_data = download_data.reset_index(drop = True)
download_data = download_data.loc[:, ['state', 'description', 'link']]
# download_data

In [55]:
#download files
# Directory where the zip files will be saved
save_directory = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads"

# Ensure the directory exists
os.makedirs(save_directory, exist_ok=True)

# Loop through URLs and download the zip files
for i,(a,b) in enumerate(zip(download_data['state'], download_data['link'])):

    state_directory = os.path.join(save_directory, a)
    
    try:
        os.makedirs(state_directory, exist_ok=True)  # Create a folder for each state
    except FileExistsError:
        print("directory already exists")
        pass

    file = b.split('latest/',1)[-1].split('_csv',1)[0]

    try:
        file_name = f'{file}.zip'
        file_path = os.path.join(state_directory, file_name)
        
        print(f"Downloading {file_name}...")
        response = requests.get(b, stream=True)
        response.raise_for_status()  # Check for HTTP errors
        
        # Write the content to a file
        with open(file_path, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        
        print(f"{file_name} downloaded successfully to {file_path}.")
    except Exception as e:
        print(f"Error downloading {b}: {e}")

AL_2024rs.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads\Alabama\AL_2024rs.zip.
AL_2025rs.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads\Alabama\AL_2025rs.zip.
AK_33.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads\Alaska\AK_33.zip.
AK_34.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads\Alaska\AK_34.zip.
AZ_56th-2nd-regular.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads\Arizona\AZ_56th-2nd-regular.zip.
AZ_57th-1st-regular.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads\Arizona\AZ_57th-1st-regular.zip.
AZ_55th-2nd-regular.zip downloaded successfully to C:\Users\clutz\OneDrive - THE HUN

In [67]:

rootdir = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads'
for subdir, dirs, files in os.walk(rootdir):
    # print(subdir)
    # print(dirs)
    for file in files:
        print(file)
        # print(os.path.join(subdir, file))
        print(os.getcwd())

AL_2024rs.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AL_2025rs.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AK_33.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AK_34.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AZ_55th-2nd-regular.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AZ_56th-2nd-regular.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AZ_57th-1st-regular.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AR_2024F.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AR_2024S2.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
AR_2025.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
CA_20232024%20Special%20Session%201.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
CA_20232024%20Special%20Session%202.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
CA_20232024.zip
c:\Users\clutz\Projects\archive\openstates bill pulling
CA_20252026%20Spe

In [ ]:
#unzip files
import zipfile
import glob
rootdir = r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Documents\Data\bill_data\Bulk Downloads'
os.chdir(rootdir)
zip_files = glob.glob('**/*.zip')

print(zip_files)

for i,file in enumerate(zip_files):
    print("#################################")

    print(file)
    
    if i == 0:
        continue


    print(file)
    folder = file.replace('.zip', '').strip()
    zip_folder = os.path.join(subdir, folder)

    # print(zip_folder)

    try:
        os.makedirs(zip_folder, exist_ok=True)  # Create a folder for each state
    except FileExistsError:
        print("directory already exists")
        pass

    path = os.path.join(rootdir, file)
    try:
        with zipfile.ZipFile(path, 'r') as zip_ref:
                zip_ref.extractall(zip_folder)

        print(f'{file} downloaded')
    except:
            print('it didnt download')
            print(path)


In [114]:
bill_dfs = []
for subdir, dirs, files in os.walk(rootdir):
    # print('____________________')
    # print(subdir)
    # print(dirs)
    # print(files)
    for file in files:
        if re.search(r'bills\.csv', str(file)) and 'related' not in str(file):
            # print(file)
            path = os.path.join(subdir, file)
            df = pd.read_csv(path)
            bill_dfs.append(df)


        

In [ ]:

all_bills = pd.concat(bill_dfs)
all_bills = all_bills.reset_index(drop = True)
all_bills


In [116]:
test_df = bill_dfs[0]
test_df

,id,identifier,title,classification,subject,session_identifier,jurisdiction,organization_classification
0,ocd-bill/a4e860c9-2a2a-4fb7-a972-a7e0a2d0d820,HR 27,Agricultural Equipment Manufacturers for Right...,['resolution'],['Resolutions'],2024rs,Alabama,lower
1,ocd-bill/f33c01d7-5ea9-4e87-b4f8-54fe415d197e,HJR 16,"Easton Fothergill, commended",['resolution'],['Resolutions'],2024rs,Alabama,lower
2,ocd-bill/0586072e-3cfd-492d-8aa9-84eb9626d496,HJR 17,COMMENDING MR. WILL REICHARD,['resolution'],['COMMENDING MR. WILL REICHARD'],2024rs,Alabama,lower
3,ocd-bill/2ff8fa9c-796e-4b9b-89dd-824332fe3bc4,SB 79,"Occupational Licensing Boards, consideration o...",['bill'],['Occupational Licensing Boards'],2024rs,Alabama,upper
4,ocd-bill/595bce3c-5764-4593-b5ee-54e0f16164e5,SR 33,SPECIAL ORDER CALENDAR,['resolution'],['SPECIAL ORDER CALENDAR'],2024rs,Alabama,upper
...,...,...,...,...,...,...,...,...
1198,ocd-bill/ba2da17a-3e6d-4ebc-8514-a3239be0c195,HB 286,"Junior Achievement of Alabama, Inc., exempt fr...",['bill'],['Taxation & Revenue'],2024rs,Alabama,lower
1199,ocd-bill/3a5d7ea6-14f2-450b-8d5b-be9d043ca140,HB 178,To exempt food pantries in Cullman County from...,['bill'],['Taxation & Revenue'],2024rs,Alabama,lower
1200,ocd-bill/b67efcf9-0aaf-4938-95d0-3f96f08681de,SB 108,"Driver Licenses, invalidates any class of ou...",['bill'],['Motor Vehicles & Traffic'],2024rs,Alabama,upper
1201,ocd-bill/4f3a7528-e2ae-4b69-b1c5-1b8cbc31475e,SB 18,"Commencement of actions, Scouts' Honor Bill, e...",['bill'],['Crimes & Offenses'],2024rs,Alabama,upper


In [ ]:
for file in bill_files:
    print(file.split('\\')[-1])

    df = pd.read_csv(file)

AL_2024rs_bills.csv
AL_2025rs_bills.csv
AK_33_bills.csv
AK_34_bills.csv
AZ_55th-2nd-regular_bills.csv
AZ_56th-2nd-regular_bills.csv
AZ_57th-1st-regular_bills.csv
AR_2024F_bills.csv
AR_2024S2_bills.csv
AR_2025_bills.csv
CA_20232024_bills.csv
CA_20232024 Special Session 1_bills.csv
CA_20232024 Special Session 2_bills.csv
CA_20252026_bills.csv
CA_20252026 Special Session 1_bills.csv
CO_2024A_bills.csv
CO_2024B_bills.csv
CO_2025A_bills.csv
CT_2024_bills.csv
CT_2025_bills.csv
DE_152_bills.csv
DE_153_bills.csv
FL_2024_bills.csv
FL_2025_bills.csv
GA_2023_24_bills.csv
GA_2025_26_bills.csv
HI_2024_bills.csv
HI_2025_bills.csv
ID_2024_bills.csv
ID_2025_bills.csv
IL_97th_bills.csv
IL_98th_bills.csv
IL_99th_bills.csv
IN_2024_bills.csv
IN_2025_bills.csv
IA_2023-2024_bills.csv
IA_2025-2026_bills.csv
KS_2023-2024_bills.csv
KS_2025-2026_bills.csv
KY_2024RS_bills.csv
KY_2025RS_bills.csv
LA_2024_bills.csv
LA_2024s1_bills.csv
LA_2024s2_bills.csv
LA_2024s3_bills.csv
ME_131_bills.csv
ME_132_bills.csv
MD_202

# END

In [ ]:

for i,j in enumerate(grouped['fetch_1_25']):
    if len(j) == 1:
        print(grouped.loc[i,'state'])


In [ ]:



response = requests.get(url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

tags = soup.find_all('ul')

# Extract content between the tags
for idx, tag in enumerate(tags):

    print(tag)
    # print(f"Content in tag {idx + 1}: {tag.text}")